<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/HMM_Model_Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance
!pip install pandas_ta
!pip install hmmlearn

  Using cached hmmlearn-0.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
Using cached hmmlearn-0.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (165 kB)


In [4]:
import sys
import os

# Point to your Drive folder
drive_lib = '/content/drive/My Drive/python_libs'
if drive_lib not in sys.path:
    sys.path.insert(0, drive_lib)

try:
    import pandas_ta as ta
    import pandas as pd
    import numpy as np
    print("Success! Environment is ready for 2022 analysis.")
    print(f"Pandas Version: {pd.__version__}")
except Exception as e:
    print(f"Error: {e}")

Success! Environment is ready for 2022 analysis.
Pandas Version: 3.0.0


In [2]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np
from hmmlearn.hmm import GaussianHMM

# 1. Configuration
SYMBOLS = ["TSLA", "SPY", "QQQ", "NVDA", "AAPL"] # Scaled down for example speed
HMM_WINDOW = 60  # Days of lookback the HMM uses to 'learn' the current regime
ST_LENGTH = 10
ST_MULT = 2

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="2y", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Rolling HMM & Strategy Logic
all_trades_list = []
print("Running HMM Backtest (this may take a moment)...")

for symbol, df in data_cache.items():
    # --- Feature Engineering for HMM ---
    df['returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['range'] = (df['High'] - df['Low']) / df['Close']
    df = df.dropna().copy()

    # Pre-calculate Strategy Indicators
    st = ta.supertrend(df['High'], df['Low'], df['Close'], length=ST_LENGTH, multiplier=ST_MULT)
    stc = ta.stc(df['Close'])
    rsi = ta.rsi(df['Close'], length=14)
    bb = ta.bbands(df['Close'], length=20)

    # --- FIX: Dynamic Column Mapping ---
    # Find the SuperTrend 'Floor' column (usually starts with SUPERT_)
    st_floor_col = [c for c in st.columns if c.startswith('SUPERT_') and 'd_' not in c.lower()]
    if not st_floor_col:
        print(f"Skipping {symbol}: SuperTrend calculation failed.")
        continue

    # Map values to the main dataframe
    df['st_floor'] = st[st_floor_col[0]]
    df['stc'] = stc.iloc[:, 0] # STC returns a df, take the first column
    df['rsi'] = rsi

    # Find the Bollinger Lower Band column (starts with BBL_)
    bbl_col = [c for c in bb.columns if c.startswith('BBL_')]
    if bbl_col:
        df['bb_lower'] = bb[bbl_col[0]]

    # --- Rolling HMM Regime Detection ---
    regimes = []
    # We start at HMM_WINDOW to ensure the model has enough data to train
    for i in range(len(df)):
        if i < HMM_WINDOW:
            regimes.append(None)
            continue

        # --- FIX: Create a writable copy of the slice ---
        train_slice = df[['returns', 'range']].iloc[i-HMM_WINDOW:i].values.copy()

        # Now you can safely add the noise
        train_slice += np.random.normal(0, 1e-6, train_slice.shape)

        # REFINEMENT 2: Increase n_iter and set a convergence tolerance
        model = GaussianHMM(
            n_components=2,
            covariance_type="diag",
            n_iter=1000,        # Increased for better convergence
            tol=1e-3,
            random_state=42
        )

        try:
            model.fit(train_slice)
            # Predict the most recent state
            states = model.predict(train_slice)
            curr_state = states[-1]

            # Variance labeling logic
            state_vars = [np.var(train_slice[states == j, 0]) for j in range(2)]
            trending_state = np.argmin(state_vars)

            regimes.append("TRENDING" if curr_state == trending_state else "SIDEWAYS")
        except Exception as e:
            # Fallback if the specific window is mathematically unsolvable
            regimes.append(regimes[-1] if (len(regimes) > 0 and regimes[-1]) else "SIDEWAYS")

    df['regime'] = regimes

    # --- Signal Generation ---
    # Logic 1: Trending Signal (Price > SuperTrend AND STC > 25)
    df['trend_signal'] = (df['regime'] == "TRENDING") & (df['Close'] > df['st_floor']) & (df['stc'] > 25)

    # Logic 2: Sideways Signal (RSI < 35 AND Price < Lower BB)
    df['sideways_signal'] = (df['regime'] == "SIDEWAYS") & (df['rsi'] < 35) & (df['Close'] < df['bb_lower'])

    # Combine signals
    trades = df[df['trend_signal'] | df['sideways_signal']].copy()

    if not trades.empty:
        trades['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
        trades['Symbol'] = symbol
        all_trades_list.append(trades[['Symbol', 'regime', 'Close', 'Ret_5D']])

# 4. Final Processing
master_trade_log = pd.concat(all_trades_list).reset_index()
print(f"Done! {len(master_trade_log)} trades generated using HMM Logic.")
print(master_trade_log.groupby('regime')['Ret_5D'].mean()) # View performance per regime

Fetching fresh data...


Running HMM Backtest (this may take a moment)...


Streaming output truncated to the last 5000 lines.
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:4268: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:215: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:4268: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/

Done! 155 trades generated using HMM Logic.
regime
SIDEWAYS    0.011591
TRENDING    0.002961
Name: Ret_5D, dtype: float64


In [3]:
# Sort by the best performing trades to see what worked
top_trades = master_trade_log.sort_values(by='Ret_5D', ascending=False)
print("\n--- TOP 10 PERFORMING TRADES ---")
print(top_trades.head(10))

# Save to CSV so you can open it in Excel
master_trade_log.to_csv("hmm_backtest_results.csv", index=False)
print("\nResults saved to 'hmm_backtest_results.csv'")


--- TOP 10 PERFORMING TRADES ---
Price       Date Symbol    regime       Close    Ret_5D
6     2024-12-10   TSLA  TRENDING  400.989990  0.196688
5     2024-12-09   TSLA  TRENDING  389.790009  0.187870
150   2025-04-08   AAPL  SIDEWAYS  171.671783  0.172370
100   2024-05-17   NVDA  TRENDING   92.431839  0.151278
78    2025-04-08    QQQ  SIDEWAYS  414.577881  0.100779
99    2024-05-16   NVDA  TRENDING   94.310890  0.100043
13    2025-02-11   TSLA  SIDEWAYS  328.500000  0.097595
47    2025-04-08    SPY  SIDEWAYS  492.193604  0.082843
77    2025-04-07    QQQ  SIDEWAYS  422.180695  0.079752
76    2025-04-04    QQQ  SIDEWAYS  421.164337  0.075070

Results saved to 'hmm_backtest_results.csv'


In [3]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# Define display columns for consistency
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# --- SECTION A: Best Parameter Set PER Symbol ---
# Sort by Symbol and Win Rate, then take the top 1 for each ticker
best_per_symbol = performance_summary.sort_values(
    ['Symbol', 'Win_Rate_10D', 'Avg_10D_Ret'],
    ascending=[True, False, False]
).groupby('Symbol').head(1)

print("--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---")
display(best_per_symbol[display_cols].sort_values('Win_Rate_10D', ascending=False).head(25).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))


--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
308,ASTS,19,20,3,100.0%,10.23%,100.0%,12.94%,100.0%,12.54%
188,AMZN,14,10,8,37.5%,0.95%,87.5%,2.13%,100.0%,4.26%
764,CRWV,18,20,1,100.0%,2.82%,100.0%,12.24%,100.0%,20.67%
807,CVNA,16,10,6,100.0%,25.98%,83.3%,23.02%,100.0%,27.56%
387,BA,11,10,1,100.0%,0.24%,100.0%,9.43%,100.0%,8.31%
560,BMNR,17,10,1,100.0%,6.16%,100.0%,18.79%,100.0%,6.82%
702,CRCL,13,10,1,100.0%,7.75%,100.0%,4.23%,100.0%,21.12%
2370,SPY,19,20,2,50.0%,1.25%,100.0%,3.43%,100.0%,5.05%
2293,SOFI,11,10,2,50.0%,0.95%,50.0%,13.11%,100.0%,13.19%
2268,SNDK,19,30,2,100.0%,7.70%,100.0%,13.21%,100.0%,24.64%


In [4]:
# 1. Global Aggregation: Using the Raw Trade Log
# This avoids 'average of averages' for both Win Rates AND Returns.
global_strategy_raw = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Symbol': 'count',      # Total Trade Count
    'Ret_3D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_5D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_10D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ]
})

# 2. Flatten Multi-index columns for cleaner access
global_strategy_raw.columns = [
    'Total_Trades',
    'Wins_3D', 'Avg_3D_Ret',
    'Wins_5D', 'Avg_5D_Ret',
    'Wins_10D', 'Avg_10D_Ret'
]
global_strategy_raw = global_strategy_raw.reset_index()

# 3. Recalculate Global Win Rates from raw counts
global_strategy_raw['Win_Rate_3D'] = global_strategy_raw['Wins_3D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_5D'] = global_strategy_raw['Wins_5D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_10D'] = global_strategy_raw['Wins_10D'] / global_strategy_raw['Total_Trades']

# 4. Final Formatting & Sorting
final_cols = [
    'SMA_Param', 'BB_Param', 'Total_Trades',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

global_strategy_raw = global_strategy_raw[final_cols].sort_values(by='Win_Rate_10D', ascending=False)

print("--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---")
display(global_strategy_raw.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---


,SMA_Param,BB_Param,Total_Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
26,12,10,307,50.2%,0.32%,55.0%,0.89%,58.6%,1.71%
32,14,10,398,53.0%,0.80%,54.3%,0.97%,57.3%,2.15%
23,11,10,200,52.5%,0.94%,51.5%,0.77%,56.5%,0.82%
27,12,20,1481,53.2%,0.48%,53.7%,0.50%,56.0%,1.17%
24,11,20,1649,53.2%,0.37%,52.9%,0.33%,56.0%,1.29%
25,11,30,1839,55.4%,0.48%,54.1%,0.47%,56.0%,1.44%
29,13,10,372,51.1%,0.39%,53.0%,0.75%,55.9%,1.49%
28,12,30,1714,55.4%,0.51%,54.8%,0.56%,55.9%,1.32%
33,14,20,1188,52.7%,0.53%,52.6%,0.55%,55.5%,1.24%
30,13,20,1335,52.4%,0.42%,53.0%,0.39%,55.1%,1.08%


In [5]:
# --- Symbol Drill-Down Section ---
target_symbol = "NVDA"  # Change this to any symbol from your list

print(f"--- Top 10 Parameter Combinations for {target_symbol} (Sorted by Avg_10D_Ret) ---")

# Filter the performance_summary we created above
symbol_drill_down = performance_summary[performance_summary['Symbol'] == target_symbol]

# Display the results
display(symbol_drill_down[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Parameter Combinations for NVDA (Sorted by Avg_10D_Ret) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
1881,NVDA,11,30,29,69.0%,1.15%,69.0%,1.87%,62.1%,5.76%
1878,NVDA,10,30,31,67.7%,1.15%,67.7%,1.78%,61.3%,5.45%
1884,NVDA,12,30,29,65.5%,0.88%,58.6%,1.31%,65.5%,5.03%
1876,NVDA,9,30,37,67.6%,1.42%,62.2%,1.80%,56.8%,5.01%
1890,NVDA,14,30,28,75.0%,2.21%,57.1%,1.53%,57.1%,4.75%
1887,NVDA,13,30,25,76.0%,2.07%,64.0%,2.20%,60.0%,4.67%
1891,NVDA,15,10,9,66.7%,1.43%,44.4%,1.31%,55.6%,4.65%
1883,NVDA,12,20,28,50.0%,0.01%,57.1%,1.27%,75.0%,4.63%
1880,NVDA,11,20,27,59.3%,0.67%,70.4%,1.65%,66.7%,4.56%
1873,NVDA,8,30,39,69.2%,1.73%,61.5%,1.52%,59.0%,4.45%


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Prepare the Aggregate Data (if not already done in Cell 2)
param_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': 'mean',
    'Ret_5D': 'mean',
    'Ret_10D': 'mean'
}).reset_index()

# 2. Define the horizons we want to visualize
horizons = [('3D', 'Ret_3D'), ('5D', 'Ret_5D'), ('10D', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Average Returns) ---")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_table = param_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    # Create the visualization using Pandas Styler for a clean Colab output
    print(f"\n{label} Horizon Average Return:")
    display(pivot_table.style.background_gradient(cmap='RdYlGn', axis=None)
            .format("{:.2%}")
            .set_caption(f"Average {label} Return by SMA and BB Period"))

# Optional: If you prefer a more "visual" graphical heatmap using Seaborn:
# plt.figure(figsize=(10, 6))
# sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn")
# plt.title("10D Return Surface")
# plt.show()

--- PARAMETER SURFACE HEATMAPS (Average Returns) ---

3D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.47%,0.39%,0.33%
4,0.37%,0.24%,0.20%
5,-0.09%,0.15%,0.13%
6,-0.25%,0.16%,0.22%
7,-0.25%,0.07%,0.19%
8,-0.16%,0.16%,0.31%
9,-0.17%,0.15%,0.31%
10,nan%,0.30%,0.40%
11,0.94%,0.37%,0.48%



5D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,0.87%,0.33%,0.29%
4,0.91%,0.18%,0.20%
5,-0.13%,0.08%,0.16%
6,-0.39%,0.05%,0.20%
7,-0.28%,0.08%,0.24%
8,-0.31%,0.09%,0.30%
9,-0.12%,0.18%,0.32%
10,nan%,0.32%,0.42%
11,0.77%,0.33%,0.47%



10D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,1.04%,0.97%,0.96%
4,0.89%,0.75%,0.76%
5,0.38%,0.60%,0.69%
6,0.25%,0.79%,0.89%
7,0.59%,0.98%,1.07%
8,0.85%,1.11%,1.22%
9,0.75%,1.13%,1.26%
10,nan%,1.16%,1.32%
11,0.82%,1.29%,1.44%


In [7]:
# 1. Prepare the Win Rate Aggregate Data
# We calculate the mean of the boolean (Return > 0) to get the percentage of wins
win_rate_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': lambda x: (x > 0).mean(),
    'Ret_5D': lambda x: (x > 0).mean(),
    'Ret_10D': lambda x: (x > 0).mean()
}).reset_index()

# 2. Define the horizons for visualization
horizons = [('3-Day', 'Ret_3D'), ('5-Day', 'Ret_5D'), ('10-Day', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---")
print("Focus: Probability of a positive return across the portfolio.")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_win_rate = win_rate_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    print(f"\n{label} Win Rate Surface:")
    # Using a 0.5 (50%) midpoint for the gradient helps highlight 'Edge' vs 'Coin Flip'
    display(pivot_win_rate.style.background_gradient(cmap='RdYlGn', vmin=0.45, vmax=0.65, axis=None)
            .format("{:.1%}")
            .set_caption(f"{label} Win Rate by SMA and BB Period"))

--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---
Focus: Probability of a positive return across the portfolio.

3-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,51.1%,52.7%,52.5%
4,50.1%,52.0%,52.0%
5,48.7%,51.4%,51.5%
6,47.8%,51.5%,52.3%
7,47.5%,51.5%,52.4%
8,47.6%,51.9%,53.5%
9,47.7%,51.7%,53.8%
10,nan%,52.9%,54.8%
11,52.5%,53.2%,55.4%



5-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,50.0%,51.5%,51.6%
4,49.8%,50.8%,51.2%
5,48.2%,49.9%,50.8%
6,48.0%,50.8%,51.6%
7,48.4%,51.2%,51.9%
8,48.7%,52.0%,52.6%
9,49.5%,52.1%,53.0%
10,nan%,53.2%,53.9%
11,51.5%,52.9%,54.1%



10-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,52.8%,53.2%,53.0%
4,51.6%,52.2%,52.3%
5,50.4%,51.5%,51.8%
6,49.9%,52.1%,52.4%
7,50.4%,53.0%,53.1%
8,51.6%,53.7%,53.7%
9,52.0%,54.6%,54.8%
10,nan%,54.9%,55.0%
11,56.5%,56.0%,56.0%


In [8]:
# 1. Create a copy for export to keep the original data intact
export_df = global_strategy_raw.copy()

# 2. Round all numerical columns to 4 decimal places
# This ensures that Win Rates (0.6543 -> 0.65) and Returns (0.0123 -> 0.01) are consistent
export_df = export_df.round(4)

# 3. Rename columns for the final report
export_df = export_df.rename(columns={
    'SMA_Param': 'SMA',
    'BB_Param': 'BB',
    'Total_Trades': 'Trades'
})

# 4. Export to CSV string
# float_format="%.4f" is a safety measure to ensure trailing zeros are kept (e.g., 0.50)
csv_output = export_df.to_csv(index=False, float_format="%.4f")

print("--- COPY AND PASTE THIS TO GEMINI ---")
print(csv_output)

--- COPY AND PASTE THIS TO GEMINI ---
SMA,BB,Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
12,10,307,0.5016,0.0032,0.5505,0.0089,0.5863,0.0171
14,10,398,0.5302,0.0080,0.5427,0.0097,0.5729,0.0215
11,10,200,0.5250,0.0094,0.5150,0.0077,0.5650,0.0082
12,20,1481,0.5321,0.0048,0.5375,0.0050,0.5604,0.0117
11,20,1649,0.5318,0.0037,0.5288,0.0033,0.5603,0.0129
11,30,1839,0.5541,0.0048,0.5405,0.0047,0.5601,0.0144
13,10,372,0.5108,0.0039,0.5296,0.0075,0.5591,0.0149
12,30,1714,0.5543,0.0051,0.5478,0.0056,0.5589,0.0132
14,20,1188,0.5269,0.0053,0.5261,0.0055,0.5547,0.0124
13,20,1335,0.5236,0.0042,0.5296,0.0039,0.5513,0.0108
10,30,1946,0.5483,0.0040,0.5391,0.0042,0.5504,0.0132
13,30,1611,0.5493,0.0048,0.5438,0.0050,0.5500,0.0123
14,30,1536,0.5462,0.0055,0.5391,0.0058,0.5495,0.0127
10,20,1784,0.5286,0.0030,0.5320,0.0032,0.5488,0.0116
9,30,2084,0.5379,0.0031,0.5298,0.0032,0.5485,0.0126
9,20,1957,0.5171,0.0015,0.5212,0.0018,0.5457,0.0113
15,10,425,0.4988,0.0034,0.5294,0.00

In [9]:
# 1. Aggregate Advanced Risk Metrics from Raw Data
# We focus on the 10-Day window as it showed the highest potential for spreads
stress_test = master_trade_log.groupby(['SMA_Param', 'BB_Param'])['Ret_10D'].agg([
    ('Trades', 'count'),
    ('Avg_Ret', 'mean'),
    ('Median_Ret', 'median'),
    ('Std_Dev', 'std'),
    ('Win_Rate', lambda x: (x > 0).mean())
]).reset_index()

# 2. Calculate Expectancy Score (Assuming 1:1 Risk/Reward Ratio)
# Formula: (WinRate * 1) - (LossRate * 1)
# A score of 0.10 means you expect to make $0.10 for every $1.00 risked over time.
stress_test['Expectancy_Score'] = (stress_test['Win_Rate'] * 2) - 1

# 3. Sort by Expectancy Score
stress_test = stress_test.sort_values('Expectancy_Score', ascending=False)

print("--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---")
print("Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.")

display(stress_test.head(10).style.format({
    'Avg_Ret': '{:.2%}',
    'Median_Ret': '{:.2%}',
    'Std_Dev': '{:.2%}',
    'Win_Rate': '{:.1%}',
    'Expectancy_Score': '{:.3f}'
}).background_gradient(subset=['Expectancy_Score'], cmap='Greens'))

--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---
Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.


,SMA_Param,BB_Param,Trades,Avg_Ret,Median_Ret,Std_Dev,Win_Rate,Expectancy_Score
26,12,10,304,1.71%,1.15%,10.83%,58.6%,0.173
32,14,10,393,2.15%,0.88%,10.88%,57.3%,0.146
23,11,10,199,0.82%,0.93%,9.81%,56.5%,0.130
27,12,20,1475,1.17%,0.91%,11.17%,56.0%,0.121
24,11,20,1641,1.29%,0.87%,11.60%,56.0%,0.121
25,11,30,1828,1.44%,0.96%,11.64%,56.0%,0.120
29,13,10,368,1.49%,0.80%,10.97%,55.9%,0.118
28,12,30,1706,1.32%,0.97%,11.16%,55.9%,0.118
33,14,20,1183,1.24%,0.89%,11.36%,55.5%,0.109
30,13,20,1330,1.08%,0.87%,11.48%,55.1%,0.103
